In [23]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset
import torch
import torchvision
import torchaudio
import librosa
import soundfile as sf
import numpy as np

In [24]:
# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2")
model.config.forced_decoder_ids = None

In [ ]:
# load dummy dataset and read audio files
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
print(np.shape(ds))
print(ds[0])
sample = ds[0]["audio"]
input_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features 

In [ ]:
# generate token ids (encoded text)
predicted_ids = model.generate(input_features)
# decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True) # set true or false to dictate whether tokens should be set

In [ ]:
print(transcription)


In [33]:
# Load and resample the audio file
audio_path = "./recording1.wav"
waveform, original_sample_rate = torchaudio.load(audio_path)
resampler = torchaudio.transforms.Resample(original_sample_rate, 16000)
waveform_resampled = resampler(waveform)

# Process the resampled audio using the Whisper processor
input_features = processor(waveform_resampled.numpy(), return_tensors="pt").input_features

# Generate token ids (encoded text)
predicted_ids = model.generate(input_features)

# Decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
print("Transcription:", transcription)







It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Transcription:  Hi, my name is Kevin and I'm writing this software to test to see if my ASR


In [32]:

# import required libraries
import sounddevice as sd
from scipy.io.wavfile import write
import wavio as wv

# Sampling frequency
freq = 44100
 
# Recording duration
duration = 5
 
# Start recorder with the given values
# of duration and sample frequency
recording = sd.rec(int(duration * freq),
                   samplerate=freq, channels=2)
 
# Record audio for the given number of seconds
sd.wait()
 
# This will convert the NumPy array to an audio
# file with the given sampling frequency
#write("recording0.wav", freq, recording)
 
# Convert the NumPy array to audio file
wv.write("recording1.wav", recording, freq, sampwidth=2)